In [1]:
import bokeh
from bokeh.plotting import figure, show, output_notebook
from bokeh.charts import Scatter, Line, color
from bokeh import palettes
from bokeh.layouts import row
import colorcet
output_notebook()

import numpy as np
import pandas as pd
import os
import math
import time
from IPython.display import clear_output

colors = palettes.Paired12
basedirs = ['results',
            '/Volumes/dev/code/structure-experiments/results',
            '/Volumes/speedy/code/structure-experiments/results',
            '/mnt/nyu/structure-experiments/results',
            '/mnt/speedy/code/structure-experiments/results']

tooltips = [
    ('Experiment', '@experiment'),
    ('Step', '@step'),
    ('Loss', '@loss'),
    ('Prior divergence', '@priordivergence'),
    ('Transition divergence', '@transdivergence'),
]

/Users/willw/anaconda/lib/python3.5/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
The bokeh.charts API has moved to a separate 'bkcharts' package.

This compatibility shim will remain until Bokeh 1.0 is released.
After that, if you want to use this API you will have to install
the bkcharts package explicitly.

  warn(message)


Loading BokehJS ...

In [2]:
def sparsify(steps, values, amount):
    if amount < 2: return steps, values
    sparse_steps = []
    sparse_values = []

    for i in range(min(len(steps), len(values))):
        if i % amount == 0:
            sparse_values.append(values[i])
            sparse_steps.append(steps[i])
    return np.array(sparse_steps), np.array(sparse_values)

In [3]:
def load_result_df(experiment, smoothing=1, max_points=500, upto=None):
    for basedir in basedirs:
        df = pd.DataFrame()
        try:
            path = os.path.join(basedir, experiment, 'results.csv')
            df = pd.read_csv(path, error_bad_lines=False)
            if len(df) > 0:
                break
        except Exception as e:
#             print("Error in parsing experiment {} from basedir {}:".format(experiment, basedir), e)
            continue

    numeric_columns = list(df.columns)
    df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors="coerce")

    if upto is not None:
        df = pd.DataFrame(df.query("step<{}".format(upto)))
    
    for col in numeric_columns:
        df[col] = df[col].rolling(smoothing, center=True).mean()
    
    for col in df:
        df["".join(col.split()).lower()] = df[col]
    df['experiment'] = pd.Series([experiment] * len(df), index=df.index)
    
    every_nth = int(len(df) / max_points) + 1
    return df.iloc[::every_nth, :]

def make_dataframe(results, smoothing=1, max_points=500, upto=None):
    data = pd.DataFrame()
    for experiment in results:
        single_df = load_result_df(experiment, smoothing, max_points, upto)
        data = data.append(single_df, ignore_index=True)
    return data

In [49]:
title = "mmnist bce"
networks = [
#     "work-mmnist-bce",
    "work-mmnist-bce-bn",
#     "work-mmnist-bce-bn-trans_1markov",
#     "work-mmnist-bce-bn-shallowtrans",
    "work-mmnist-bce-bn-2",
    "work-mmnist-bce-ld10",
    "cmmnist_bce",
]

data = make_dataframe(networks, smoothing=1, max_points=500, upto=4e6)

# palette = palettes.Spectral11[:5] + palettes.Spectral11[6:]
spaced_palette = palettes.linear_palette(colorcet.rainbow, len(networks))
# spaced_palette = palettes.viridis(len(networks))

plots = []
for facet in ['nll', 'priordivergence', 'transdivergence']:
    plot = Line(data, x='step', y=facet,
                title=title + ", " + facet,
                color=color(columns='experiment', palette=spaced_palette),
                tools='pan,wheel_zoom,box_zoom,save,reset', 
                active_scroll="wheel_zoom",
                tooltips=tooltips,
                plot_height=300,
                legend='top_right',)
    plots.append(plot)
    show(row(children=[plot], responsive=True))

In [5]:
title = "mmnist normal loss"
networks = [
#     "work-mmnist-var0.01",
#     "work-mmnist-var0.05",
#     "work-mmnist-var0.1",
#     "work-mmnist-var0.2",
#     "work-mmnist-ld10-var0.1",
#     "work-mmnist-ld10-var0.2",
    
#     "work-mmnist-1markov-ld10-var0.1",
#     "work-mmnist-1markov-ld10-var0.2",
    
    "work-mmnist-1markov-ld20-var0.25",
#     "work-mmnist-1markov-ld20-var0.3",
    
    "work-l1-ld40-var0.25-kl0.5",
    "work-l1-ld40-var0.25-kl2",
    "work-l1-ld40-var0.25-kl5",
    "work-l1-ld40-var0.25",
    
    
#     "cmmnist_bce",
]

data = make_dataframe(networks, smoothing=1, max_points=500)

# palette = palettes.Spectral11[:5] + palettes.Spectral11[6:]
spaced_palette = palettes.linear_palette(colorcet.rainbow, len(networks))
# spaced_palette = palettes.viridis(len(networks))

plots = []
for facet in ['nll', 'priordivergence', 'transdivergence']:
    plot = Line(data, x='step', y=facet,
                title=title + ", " + facet,
                color=color(columns='experiment', palette=spaced_palette),
                tools='pan,wheel_zoom,box_zoom,save,reset', 
                active_scroll="wheel_zoom",
                tooltips=tooltips,
                plot_height=300,
                legend='top_right',)
    plots.append(plot)
    show(row(children=[plot], responsive=True))

In [7]:
title = "5thave normal loss"
networks = [
    "work-5thave-ld25-l8-var0.3",
    "work-5thave-ld25-l8-var0.2",
    "work-5thave-ld25-l8-var0.1",
    "work-5thave-ld25-l8-var0.05",

#     "work-5thave-ld10-l8-var0.3",
#     "work-5thave-ld10-l8-var0.2",
#     "work-5thave-ld10-l8-var0.1",
#     "work-5thave-ld10-l8-var0.05",
    
    "work-5thave-ld25-l8-nobottleneck-var0.05",
    "work-5thave-ld25-l8-nobottleneck-var0.08",
    "work-5thave-ld25-l8-nobottleneck-var0.1",
    "work-5thave-ld25-l8-nobottleneck-var0.2",
    "work-5thave-ld25-l8-nobottleneck-var0.3",
    
    
    "5thave_128_dcgan_nodecay_z8_var0.2_6e4_1",
]

data = make_dataframe(networks, smoothing=1, max_points=500)

# palette = palettes.Spectral11[:5] + palettes.Spectral11[6:]
spaced_palette = palettes.linear_palette(colorcet.rainbow, len(networks))
# spaced_palette = palettes.viridis(len(networks))

plots = []
for facet in ['nll', 'priordivergence', 'transdivergence']:
    plot = Line(data, x='step', y=facet,
                title=title + ", " + facet,
                color=color(columns='experiment', palette=spaced_palette),
                tools='pan,wheel_zoom,box_zoom,save,reset', 
                active_scroll="wheel_zoom",
                tooltips=tooltips,
                plot_height=300,
                legend='top_right',)
    plots.append(plot)
    show(row(children=[plot], responsive=True))